In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import skew
import os
import csv
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [2]:
# 데이터 불러오기 (csv, xlsx)

df = pd.read_csv("C:/ai/DSA_features.csv")
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,8.280854,34.1980,-2.9038,28.080803,5.299132,1.350075,-1.491537,11.2240,-11.65100,14.670334,...,0.200829,-0.040701,0.297666,0.708480,-0.117430,4.135451e-02,0.203358,-0.310022,basketBall,p8
9116,9.591118,51.6970,-3.4129,35.722025,5.976791,2.981144,0.086304,6.9951,-11.76400,5.329897,...,0.148745,-0.266377,0.224716,0.554670,-0.250950,3.355704e-02,0.183186,-0.736410,basketBall,p8
9117,9.599113,27.9300,-1.0765,48.850886,6.989341,0.449237,-0.728367,3.7801,-8.36910,5.683022,...,0.310748,-0.009505,-0.237786,0.088854,-0.477260,2.026107e-02,0.142341,0.668438,basketBall,p8
9118,9.692482,72.7820,-2.6734,59.378336,7.705734,4.491114,-0.582724,6.1216,-8.85710,4.162963,...,0.156493,0.050624,0.533023,0.677800,0.055941,1.356379e-02,0.116464,-1.482489,basketBall,p8


In [3]:
for i in df.columns:
    # LabelEncoder 객체 생성
    label_encoder = LabelEncoder()
    # 'label' 컬럼을 숫자로 변환
    df[i] = LabelEncoder().fit_transform(df[i])
df

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,1824,1201,6229,2129,2129,2972,8548,3588,8343,1646,...,674,6839,3211,1960,4164,877,877,5593,12,0
1,1829,1206,6365,2027,2027,5743,8590,3641,8362,530,...,814,2752,3207,1961,4154,912,912,7434,12,0
2,1819,1179,6364,1674,1674,3636,8591,3640,8368,227,...,578,4013,3214,1969,4172,774,774,5657,12,0
3,1793,1184,6240,1769,1769,1969,8606,3853,8366,1632,...,459,5532,3248,2002,4205,1016,1016,6906,12,0
4,1768,1194,6354,1311,1311,9052,8632,3717,8378,372,...,742,3889,3262,2007,4215,886,886,7001,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,2172,6784,1403,7972,7972,8502,1765,8300,390,8908,...,7907,4951,6862,8180,3669,8798,8798,2887,1,7
9116,7405,7298,1247,8057,8057,9105,6651,7938,383,8108,...,6255,2928,6063,7189,2863,8683,8683,945,1,7
9117,7464,6452,1740,8153,8153,5305,3110,7080,1089,8153,...,9006,5238,1772,3130,1070,8389,8389,8299,1,7
9118,8146,7476,1433,8208,8208,9118,3545,7811,896,7908,...,6510,5814,8280,7988,5296,7829,7829,137,1,7


In [4]:
X=df.drop('activity',axis=1)
y=df['activity']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=0)
X_train = np.expand_dims(X_train, axis=-1)
print(X.shape)
print(y.shape)

(9120, 271)
(9120,)


In [5]:
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(X_train)
#X_train_encoded = tokenizer.texts_to_sequences(X_train)
#X_train_padded = pad_sequences(X_train_encoded, maxlen = max_len)
#max_len = 3
#print("훈련 데이터의 크기(shape):", X_train_padded.shape)

In [6]:
from keras import layers
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [7]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [17]:
#input_shape = X_train_padded.shape
input_shape = X_train.shape[1:]
print("input: {}".format(input_shape))

model = build_model(
    input_shape,
    head_size=16,
    num_heads=3,
    ff_dim=3,
    num_transformer_blocks=3,
    mlp_units=[32],
    mlp_dropout=0.2,
    dropout=0.1,)

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["categorical_accuracy"],)
model.summary()

input: (271, 1)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 271, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 271, 1)    │        337 │ input_layer_4[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_41          │ (None, 271, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 271, 1)    │          2 │ dropout_41[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 271, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_24 (Conv1D)  │ (None, 271, 3)    │          6 │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_42          │ (None, 271, 3)    │          0 │ conv1d_24[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_25 (Conv1D)  │ (None, 271, 1)    │          4 │ dropout_42[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 271, 1)    │          2 │ conv1d_25[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_25 (Add)        │ (None, 271, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_24[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 271, 1)    │        337 │ add_25[0][0],     │
│ (MultiHeadAttentio… │                   │            │ add_25[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_44          │ (None, 271, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 271, 1)    │          2 │ dropout_44[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_26 (Add)        │ (None, 271, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_25[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_26 (Conv1D)  │ (None, 271, 3)    │          6 │ add_26[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_45          │ (None, 271, 3)    │          0 │ conv1d_26[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_27 (Conv1D)  │ (None, 271, 1)    │          4 │ dropout_45[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 271, 1)    │          2 │ conv1d_27[0][0]   │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 1,150 (4.49 KB)

 Trainable params: 1,150 (4.49 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=256,
    callbacks=callbacks,
)

#model.evaluate(x_test, y_test, verbose=1)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 510ms/step - categorical_accuracy: 1.0000 - loss: 1.0908e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 484ms/step - categorical_accuracy: 1.0000 - loss: 1.0898e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 483ms/step - categorical_accuracy: 1.0000 - loss: 1.0868e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 484ms/step - categorical_accuracy: 1.0000 - loss: 1.0835e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 486ms/step - categorical_accuracy: 1.0000 - loss: 1.0917e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 485ms/step - categorical_accuracy: 1.0000 - loss: 1.0834e-06 - val_categorical_accuracy: 1.0000 - val_loss: 1.0696e-06
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━

In [23]:
X_test = np.expand_dims(X_test, axis=-1)
model.evaluate(X_test, y_test, verbose=1)

72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - categorical_accuracy: 1.0000 - loss: 1.0564e-06


[1.0521789590711705e-06, 1.0]